### Complete Tcash transactions

In this notebook we will reconstruct the complete tornado transactions df. 

we found that when running these querys:

#### Withdrawals 
SELECT * FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE `to_address` 
IN (SELECT `address` FROM `tornado_cash_transactions.tornadocontracts`) AND SUBSTR(`input`, 1, 10) = "0x21a0adb6";

#### Deposits
SELECT * FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE `to_address` 
IN (SELECT `address` FROM `tornado_cash_transactions.tornadocontracts`) AND SUBSTR(`input`, 1, 10) = "0xb214faa5";

We were missing every transaction (withdrawl and deposit) that was made **through** the Tcash proxy. By asking for the transactions that have a t_cash_contract_address in the "to_address" field, we where loosing the ones that had the "proxy_address" in that field.

For example, these could be a few rows of the transaction table we really need:

```
Wallet -> Tcash_1_eth_contract

Wallet -> Tcash_proxy

Wallet -> Tcash_10_eth_contract

..

Wallet -> Tcash_proxy
```

To solve this, we need to use the *traces* table. In this table we will find every transaction made from a wallet, but also the ones executed by the internal transaction executed by the ethereum vm (for example, executing a function of a smart contract). So now we will be able to find every transaction interacting with the Tcash_contracts, including the Tcash proxy ones.

Running this query:

INSERT `tornado_transactions.traces` SELECT * FROM `bigquery-public-data.crypto_ethereum.traces` WHERE `to_address` 
IN ( SELECT `address` FROM `tornado_transactions.tornadocontracts` )) 
AND SUBSTR(`input`, 1, 10) IN ("0xb214faa5", "0x21a0adb6");
    
We will get a table with this kind of transactions:

```
Wallet      -> Tcash_contract

Tcash_proxy -> Tcash_contract

Wallet      -> Tcash_contract

Tcash_proxy -> Tcash_contract

Tcash_proxy -> Tcash_contract

...

Tcash_proxy -> Tcash_contract
```

And we can reconstruct the transaction table by running:

SELECT * FROM `bigquery-public-data.crypto_ethereum.transactions`
WHERE `hash` IN ( SELECT `transaction_hash` FROM `tornado_transactions.traces`);

#### Decode input data 

Then the last thing to do is to decode the input data from those transaction (the ones going through the TCash proxy) to be able to know the pool (Tcash contract) that will be interacting in the internal transaction.

That is what we are going to do in this notebook.

In [1]:
import pandas as pd
from web3 import Web3
from tqdm import tqdm
tqdm.pandas()

In [2]:
tornado_traces_df = pd.read_csv("../data/tornado_traces.csv", low_memory=False)
tornado_transactions_df = pd.read_csv("../data/tornado_transactions.csv")
tornado_addresses_df = pd.read_csv("../data/tornadocontracts_abi.csv",names=["address", "token", "value", "name","abi"],sep="|")

#### First, we divide the traces by deposit and withdrawl tx

In [3]:
deposit_traces_df = tornado_traces_df[tornado_traces_df["input"].str[:10] == "0xb214faa5"]

In [4]:
withdraw_traces_df = tornado_traces_df[tornado_traces_df["input"].str[:10] == "0x21a0adb6"]

#### Then we get the deposit and withdrawls transactions by linking the hashes 

In [5]:
deposit_transactions_df = tornado_transactions_df[tornado_transactions_df["hash"].isin(deposit_traces_df["transaction_hash"])]

In [6]:
withdraw_transactions_df = tornado_transactions_df[tornado_transactions_df["hash"].isin(withdraw_traces_df["transaction_hash"])]

In [7]:
deposit_traces_df.shape[0] + withdraw_traces_df.shape[0] == tornado_traces_df.shape[0]

True

In [8]:
deposit_transactions_df.shape[0] + withdraw_transactions_df.shape[0] == tornado_transactions_df.shape[0]

True

#### Filtering the deposits by the ones that have the Tcash proxy address:

- 0x722122df12d4e14e13ac3b6895a86e84145b6967 is for the current Tcash proxy
- 0x905b63fff465b9ffbf41dea908ceb12478ec7601 is the address of the "old proxy"

In [9]:
wallet_to_proxy_deposits_df = deposit_transactions_df[deposit_transactions_df["to_address"].isin(["0x722122df12d4e14e13ac3b6895a86e84145b6967", "0x905b63fff465b9ffbf41dea908ceb12478ec7601"])]

In [10]:
wallet_to_proxy_deposits_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
18,0xc0fbe9da9cad77f410cdf436a57754e475e124189ecd...,113,100,0xa5294016f13fb0015c1d22bd521eb65a123d34dc,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,1031828,24000000000,0x13d98d1300000000000000000000000023773e65ed14...,6031107,...,NaN,NaN,1,2021-07-17 19:18:27 UTC,12846082,0x0db97b6851d8bb165e4b1dc98e3e6a3bfbe08f7bc79d...,NaN,NaN,NaN,24000000000
19,0xf134cf9457448af20c643451389c38a206c34eda1888...,147,84,0x941a9e3b91e1cc015702b897c512d265fae88a9c,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,1031828,8000000000,0x13d98d1300000000000000000000000023773e65ed14...,10258686,...,NaN,NaN,1,2021-07-04 00:00:35 UTC,12757716,0xd2f57ed3b20d232ad152c3fcac7d5e8bfcafe21e6c67...,NaN,NaN,NaN,8000000000
20,0x00a5e51e0c10a36cd44c9ee1ba464a70b70f87ce1511...,1,64,0xcf8d8e7378abdd864f49cd74071bcb35582f6315,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,1031828,10000000000,0x13d98d1300000000000000000000000007687e702b41...,7154618,...,NaN,NaN,1,2021-07-05 13:14:17 UTC,12767706,0x88c7f8729ce598a919b6eafbb9f26291408623939147...,NaN,NaN,NaN,10000000000
21,0x7ad162c4476b30ab5a1769ab8eb72df808d7bb05b656...,86,53,0x814d8daef1ddfbc8ca9bc16f00bb5b46d2eea388,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,1031828,14000000000,0x13d98d1300000000000000000000000023773e65ed14...,6545414,...,NaN,NaN,1,2021-07-18 19:40:07 UTC,12852539,0x5c52279bdf00250071b9fa5a9a30249df743eb6fb39a...,NaN,NaN,NaN,14000000000
22,0x325fec3c8aaa5e8a0c48946363df7fb226e13ab190b0...,12,206,0xb493f2cc5f0d781affe7cdbe60500e2a8d3d7b90,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,1031868,41000000000,0x13d98d1300000000000000000000000023773e65ed14...,14007947,...,NaN,NaN,1,2021-08-12 19:14:31 UTC,13012199,0xa1bc18336718139548a3452417f8f2706f332fb965e3...,NaN,NaN,NaN,41000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181260,0xb16040d97e3c75a137c2a826a2c5307d58612a0f881f...,19,382,0xb12814fdbbdc8c8f9c2741bde788813a90588be6,0x722122df12d4e14e13ac3b6895a86e84145b6967,100000000000000000000,1019350,81846727568,0x13d98d13000000000000000000000000a160cdab2256...,29897689,...,NaN,NaN,1,2021-09-07 03:08:40 UTC,13176145,0xf1b67ec807e6be55bc6995e31c3ee4bf8cb6e6610754...,8.255631e+10,1.220177e+09,2.0,81846727568
181261,0x29d727360e68b35743d73d95f344ed885e2a9e33887f...,3,173,0xdcaaa4717c46840e6bdc1e067478210b0758055c,0x722122df12d4e14e13ac3b6895a86e84145b6967,100000000000000000000,200000,53892307010,0x13d98d13000000000000000000000000a160cdab2256...,17806851,...,NaN,NaN,0,2021-09-12 17:59:59 UTC,13212438,0xa50afdfc6c73d19a886575e40b8a88e8f00d76635a6e...,6.413296e+10,1.188590e+09,2.0,53892307010
181265,0x9c5e4b8b996349237a9df035f9e38dbd69f3bb280ecc...,8,44,0x8cf22ec65677475aa41b0537b98ccee63ceca973,0x722122df12d4e14e13ac3b6895a86e84145b6967,10000000000000000000,1024570,71609345538,0x13d98d13000000000000000000000000910cbd523d97...,5911751,...,NaN,NaN,1,2021-08-29 00:56:36 UTC,13117282,0x2c3d7ef9d4e85085b72b86807278679e07b078d85996...,1.287044e+11,5.910977e+09,2.0,71609345538
181266,0xeae7c8d21bb97391f21b7f3443ac51429092e0290fe6...,189,276,0x68661550f759a41d65a8eefd3e47dd606ca76ae2,0x722122df12d4e14e13ac3b6895a86e84145b6967,100000000000000000000,1008130,51523487883,0x13d98d13000000000000000000000000a160cdab2256...,28714246,...,NaN,NaN,1,2021-10-17 01:38:32 UTC,13432575,0xc59f24dbf6573c5c17f16d08657c78dea20377230f11...,6.838974e+10,1.301710e+09,2.0,51523487883


#### We need to use web3 in order to decode the input data and get the pool that the proxy is going to interact with

In [11]:
# Just for python to understand the contract ABIs
true = True
false = False

In [12]:
tornado_proxy_df = pd.read_csv("../data/tornado_proxy_abis.csv", names=["address", "abi"],sep="|")

In [13]:
def tornado_proxy_contracts(tornado_proxys_df):
    #creates contract object given it address and abi
    w3 = Web3(Web3.HTTPProvider("https://cloudflare-eth.com"))
    proxy_contracts_dic = {}
    for index, row in tornado_proxy_df.iterrows():
        proxy_contracts_dic[row["address"]] = w3.eth.contract(address=w3.toChecksumAddress(row["address"]), abi=eval(row["abi"]))
    return proxy_contracts_dic

In [14]:
# Creates a dict linking contract addresses with it web3 object
def tornado_contracts(tornado_df):
    #creates contract object given it address and abi
    w3 = Web3(Web3.HTTPProvider("https://cloudflare-eth.com"))
    tornado_contracts_dic = {}
    for index, row in tornado_df.iterrows():
        tornado_contracts_dic[row["address"]] = w3.eth.contract(address=w3.toChecksumAddress(row["address"]), abi=eval(row["abi"]))
    return tornado_contracts_dic

In [15]:
proxy_contracts_dict = tornado_contracts(tornado_proxy_df)

In [16]:
proxy_contracts_dict

{'0x905b63fff465b9ffbf41dea908ceb12478ec7601': <web3._utils.datatypes.Contract at 0x7fd6522cd610>,
 '0x722122df12d4e14e13ac3b6895a86e84145b6967': <web3._utils.datatypes.Contract at 0x7fd61bb807c0>}

In [17]:
def tornado_address_deposits(row, tornado_contracts_dic):
    func_obj, func_params = tornado_contracts_dic[row["to_address"]].decode_function_input(row["input"])
    return func_params["_tornado"].lower()

In [18]:
wallet_to_proxy_deposits_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
18,0xc0fbe9da9cad77f410cdf436a57754e475e124189ecd...,113,100,0xa5294016f13fb0015c1d22bd521eb65a123d34dc,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,1031828,24000000000,0x13d98d1300000000000000000000000023773e65ed14...,6031107,...,NaN,NaN,1,2021-07-17 19:18:27 UTC,12846082,0x0db97b6851d8bb165e4b1dc98e3e6a3bfbe08f7bc79d...,NaN,NaN,NaN,24000000000
19,0xf134cf9457448af20c643451389c38a206c34eda1888...,147,84,0x941a9e3b91e1cc015702b897c512d265fae88a9c,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,1031828,8000000000,0x13d98d1300000000000000000000000023773e65ed14...,10258686,...,NaN,NaN,1,2021-07-04 00:00:35 UTC,12757716,0xd2f57ed3b20d232ad152c3fcac7d5e8bfcafe21e6c67...,NaN,NaN,NaN,8000000000
20,0x00a5e51e0c10a36cd44c9ee1ba464a70b70f87ce1511...,1,64,0xcf8d8e7378abdd864f49cd74071bcb35582f6315,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,1031828,10000000000,0x13d98d1300000000000000000000000007687e702b41...,7154618,...,NaN,NaN,1,2021-07-05 13:14:17 UTC,12767706,0x88c7f8729ce598a919b6eafbb9f26291408623939147...,NaN,NaN,NaN,10000000000
21,0x7ad162c4476b30ab5a1769ab8eb72df808d7bb05b656...,86,53,0x814d8daef1ddfbc8ca9bc16f00bb5b46d2eea388,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,1031828,14000000000,0x13d98d1300000000000000000000000023773e65ed14...,6545414,...,NaN,NaN,1,2021-07-18 19:40:07 UTC,12852539,0x5c52279bdf00250071b9fa5a9a30249df743eb6fb39a...,NaN,NaN,NaN,14000000000
22,0x325fec3c8aaa5e8a0c48946363df7fb226e13ab190b0...,12,206,0xb493f2cc5f0d781affe7cdbe60500e2a8d3d7b90,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,1031868,41000000000,0x13d98d1300000000000000000000000023773e65ed14...,14007947,...,NaN,NaN,1,2021-08-12 19:14:31 UTC,13012199,0xa1bc18336718139548a3452417f8f2706f332fb965e3...,NaN,NaN,NaN,41000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181260,0xb16040d97e3c75a137c2a826a2c5307d58612a0f881f...,19,382,0xb12814fdbbdc8c8f9c2741bde788813a90588be6,0x722122df12d4e14e13ac3b6895a86e84145b6967,100000000000000000000,1019350,81846727568,0x13d98d13000000000000000000000000a160cdab2256...,29897689,...,NaN,NaN,1,2021-09-07 03:08:40 UTC,13176145,0xf1b67ec807e6be55bc6995e31c3ee4bf8cb6e6610754...,8.255631e+10,1.220177e+09,2.0,81846727568
181261,0x29d727360e68b35743d73d95f344ed885e2a9e33887f...,3,173,0xdcaaa4717c46840e6bdc1e067478210b0758055c,0x722122df12d4e14e13ac3b6895a86e84145b6967,100000000000000000000,200000,53892307010,0x13d98d13000000000000000000000000a160cdab2256...,17806851,...,NaN,NaN,0,2021-09-12 17:59:59 UTC,13212438,0xa50afdfc6c73d19a886575e40b8a88e8f00d76635a6e...,6.413296e+10,1.188590e+09,2.0,53892307010
181265,0x9c5e4b8b996349237a9df035f9e38dbd69f3bb280ecc...,8,44,0x8cf22ec65677475aa41b0537b98ccee63ceca973,0x722122df12d4e14e13ac3b6895a86e84145b6967,10000000000000000000,1024570,71609345538,0x13d98d13000000000000000000000000910cbd523d97...,5911751,...,NaN,NaN,1,2021-08-29 00:56:36 UTC,13117282,0x2c3d7ef9d4e85085b72b86807278679e07b078d85996...,1.287044e+11,5.910977e+09,2.0,71609345538
181266,0xeae7c8d21bb97391f21b7f3443ac51429092e0290fe6...,189,276,0x68661550f759a41d65a8eefd3e47dd606ca76ae2,0x722122df12d4e14e13ac3b6895a86e84145b6967,100000000000000000000,1008130,51523487883,0x13d98d13000000000000000000000000a160cdab2256...,28714246,...,NaN,NaN,1,2021-10-17 01:38:32 UTC,13432575,0xc59f24dbf6573c5c17f16d08657c78dea20377230f11...,6.838974e+10,1.301710e+09,2.0,51523487883


In [19]:
#Adds the column with the Tcash contracts that the proxy is going to interact later (in a internal tx)
wallet_to_proxy_deposits_df["tornado_cash_address"] = wallet_to_proxy_deposits_df.progress_apply(lambda row: tornado_address_deposits(row, proxy_contracts_dict), axis=1)

100%|██████████| 73075/73075 [01:03<00:00, 1159.24it/s]
<ipython-input-19-bf618f0d1c46>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wallet_to_proxy_deposits_df["tornado_cash_address"] = wallet_to_proxy_deposits_df.progress_apply(lambda row: tornado_address_deposits(row, proxy_contracts_dict), axis=1)


In [20]:
wallet_to_proxy_deposits_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address
18,0xc0fbe9da9cad77f410cdf436a57754e475e124189ecd...,113,100,0xa5294016f13fb0015c1d22bd521eb65a123d34dc,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,1031828,24000000000,0x13d98d1300000000000000000000000023773e65ed14...,6031107,...,NaN,1,2021-07-17 19:18:27 UTC,12846082,0x0db97b6851d8bb165e4b1dc98e3e6a3bfbe08f7bc79d...,NaN,NaN,NaN,24000000000,0x23773e65ed146a459791799d01336db287f25334
19,0xf134cf9457448af20c643451389c38a206c34eda1888...,147,84,0x941a9e3b91e1cc015702b897c512d265fae88a9c,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,1031828,8000000000,0x13d98d1300000000000000000000000023773e65ed14...,10258686,...,NaN,1,2021-07-04 00:00:35 UTC,12757716,0xd2f57ed3b20d232ad152c3fcac7d5e8bfcafe21e6c67...,NaN,NaN,NaN,8000000000,0x23773e65ed146a459791799d01336db287f25334
20,0x00a5e51e0c10a36cd44c9ee1ba464a70b70f87ce1511...,1,64,0xcf8d8e7378abdd864f49cd74071bcb35582f6315,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,1031828,10000000000,0x13d98d1300000000000000000000000007687e702b41...,7154618,...,NaN,1,2021-07-05 13:14:17 UTC,12767706,0x88c7f8729ce598a919b6eafbb9f26291408623939147...,NaN,NaN,NaN,10000000000,0x07687e702b410fa43f4cb4af7fa097918ffd2730
21,0x7ad162c4476b30ab5a1769ab8eb72df808d7bb05b656...,86,53,0x814d8daef1ddfbc8ca9bc16f00bb5b46d2eea388,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,1031828,14000000000,0x13d98d1300000000000000000000000023773e65ed14...,6545414,...,NaN,1,2021-07-18 19:40:07 UTC,12852539,0x5c52279bdf00250071b9fa5a9a30249df743eb6fb39a...,NaN,NaN,NaN,14000000000,0x23773e65ed146a459791799d01336db287f25334
22,0x325fec3c8aaa5e8a0c48946363df7fb226e13ab190b0...,12,206,0xb493f2cc5f0d781affe7cdbe60500e2a8d3d7b90,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,1031868,41000000000,0x13d98d1300000000000000000000000023773e65ed14...,14007947,...,NaN,1,2021-08-12 19:14:31 UTC,13012199,0xa1bc18336718139548a3452417f8f2706f332fb965e3...,NaN,NaN,NaN,41000000000,0x23773e65ed146a459791799d01336db287f25334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181260,0xb16040d97e3c75a137c2a826a2c5307d58612a0f881f...,19,382,0xb12814fdbbdc8c8f9c2741bde788813a90588be6,0x722122df12d4e14e13ac3b6895a86e84145b6967,100000000000000000000,1019350,81846727568,0x13d98d13000000000000000000000000a160cdab2256...,29897689,...,NaN,1,2021-09-07 03:08:40 UTC,13176145,0xf1b67ec807e6be55bc6995e31c3ee4bf8cb6e6610754...,8.255631e+10,1.220177e+09,2.0,81846727568,0xa160cdab225685da1d56aa342ad8841c3b53f291
181261,0x29d727360e68b35743d73d95f344ed885e2a9e33887f...,3,173,0xdcaaa4717c46840e6bdc1e067478210b0758055c,0x722122df12d4e14e13ac3b6895a86e84145b6967,100000000000000000000,200000,53892307010,0x13d98d13000000000000000000000000a160cdab2256...,17806851,...,NaN,0,2021-09-12 17:59:59 UTC,13212438,0xa50afdfc6c73d19a886575e40b8a88e8f00d76635a6e...,6.413296e+10,1.188590e+09,2.0,53892307010,0xa160cdab225685da1d56aa342ad8841c3b53f291
181265,0x9c5e4b8b996349237a9df035f9e38dbd69f3bb280ecc...,8,44,0x8cf22ec65677475aa41b0537b98ccee63ceca973,0x722122df12d4e14e13ac3b6895a86e84145b6967,10000000000000000000,1024570,71609345538,0x13d98d13000000000000000000000000910cbd523d97...,5911751,...,NaN,1,2021-08-29 00:56:36 UTC,13117282,0x2c3d7ef9d4e85085b72b86807278679e07b078d85996...,1.287044e+11,5.910977e+09,2.0,71609345538,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf
181266,0xeae7c8d21bb97391f21b7f3443ac51429092e0290fe6...,189,276,0x68661550f759a41d65a8eefd3e47dd606ca76ae2,0x722122df12d4e14e13ac3b6895a86e84145b6967,100000000000000000000,1008130,51523487883,0x13d98d13000000000000000000000000a160cdab2256...,28714246,...,NaN,1,2021-10-17 01:38:32 UTC,13432575,0xc59f24dbf6573c5c17f16d08657c78dea20377230f11...,6.838974e+10,1.301710e+09,2.0,51523487883,0xa160cdab225685da1d56aa342ad

#### Now, lets add the same column to the tx that goes from a wallet directly to a TCash tontract (the ones that we already had) just to make the concatenation simpler

In [21]:
wallet_to_tcash_deposit_df = deposit_transactions_df[deposit_transactions_df["to_address"].isin(tornado_addresses_df["address"])]

In [22]:
wallet_to_tcash_deposit_df["tornado_cash_address"] = wallet_to_tcash_deposit_df["to_address"]

<ipython-input-22-01c999392726>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wallet_to_tcash_deposit_df["tornado_cash_address"] = wallet_to_tcash_deposit_df["to_address"]


In [23]:
wallet_to_tcash_deposit_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address
0,0xcf97c470a56d96625c7240d3004ae2abd9141d7ffc43...,4,10,0xb050dec5a9010f8b77a3962369b7bc737d3ed4a5,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,1200000,56000000000,0xb214faa5083d52f2c4e7a95e340a999805a8328b6546...,1677906,...,NaN,1,2020-11-02 17:47:30 UTC,11179130,0x21d86cba454fea4f7e43c68763d4cffec101b6145546...,NaN,NaN,NaN,56000000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d
1,0x2fadc4d54bf6e3ec5035f14c0096befe929998663098...,22,118,0x6e92bc493c6abbdd6a1b18416f003de2c873ab50,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,1200000,63000000000,0xb214faa502843565a5499c26b04c7a9069f1ad2621aa...,8209080,...,NaN,1,2021-01-02 23:37:35 UTC,11578054,0xd52ababd1dbbbe50277c54473c881f47fd95e78081a3...,NaN,NaN,NaN,63000000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d
10,0x7baf0a76f35c1dece97fff883aa7174454bed460b1ba...,240,171,0x8c4c44fd06f7f98f08bf6a9ca156cec9ee1f31f8,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144,0,800000,105000000000,0xb214faa50afc4880238aac4198bceddea0fbf85e9ad3...,10723996,...,NaN,0,2021-01-06 19:04:40 UTC,11602841,0x7c5f21ea2a92f5182ce8648f152b6fb3b4379096309d...,NaN,NaN,NaN,105000000000,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144
11,0x7f851ba1d7292ca565961073a1111f7408760540c6f9...,0,84,0x50b9d4af009b038506d4d84b035c451d1a3a20bc,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,100000000000000000,800000,5000000000,0xb214faa51174c586ebae02a3b4f54a0ba6a9b7aba4d7...,8465851,...,NaN,0,2020-01-05 05:19:39 UTC,9218451,0x77e0fc60906714a54697cfd9275b622ad9d2d0271c4e...,NaN,NaN,NaN,5000000000,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
12,0xbd83053f8afa7777f54a4aca6b8e112fa31b888922dc...,7,63,0x6c6e4816ecfa4481472ff88f32a3e00f2eaa95a1,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,100000000000000000,800000,30838446643,0xb214faa527a20ba920c8ae877d67ce1ebd7420dafb31...,6222489,...,NaN,0,2020-05-27 03:30:44 UTC,10145408,0x837b3482443f027f6f045644bf002243f72304686015...,NaN,NaN,NaN,30838446643,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158855,0x6ab1e8531719d3ecfa20c6deacb38ba5a5aacd0c6729...,951,92,0x5051beb48c656f965a3554f9d463719735392446,0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3,0,1200000,1000000000,0xb214faa50af0639b1f356743ca285e469675d218c33d...,6261648,...,NaN,0,2020-03-11 05:34:42 UTC,9648354,0xcf10ca1a353d5b9b7d88d547fa9f4c1ee1c45b5cc437...,NaN,NaN,NaN,1000000000,0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3
158943,0x366da9cdc6e111ae5e1a075324c9490a396329fbbd9b...,0,126,0x6720a24ad0a17e9539482782e0f79215ccec2507,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,100000000000000000,1200000,109000000000,0xb214faa52ad3cad3afeca0180a67e301d225adeabcdd...,11159704,...,NaN,1,2021-09-06 11:22:40 UTC,13171862,0xc98676eaaf8b2d4cdf5b79bc3cf67f2ee60f4b955500...,1.090000e+11,1.090000e+11,2.0,109000000000,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
178041,0xb2f1c1ab39ba8282040ced9dec071c56393fa39c1799...,0,103,0x3793644b4e9d2b7cf78eb80e1a2fa292374b8d7c,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,100000000000000000,1200000,73000000000,0xb214faa50c21560d0e113c4107c0b977cad870e178a9...,5531927,...,NaN,1,2021-08-27 09:41:57 UTC,13106753,0xb2372bbfcb4a59107ccc444adc3e08d7b907088768c7...,7.300000e+10,7.300000e+10,2.0,73000000000,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
178606,0xc83830a40ee2eaaf3c35350e980b1e2984ab874e1af9...,0,10,0x66a7534f12111b9a545a8817602ca53b6d9dba51,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,1000000000000000000,1200000,134000000000,0xb214faa5305aebaadaa62bf8dd63b8467feb949d4f41...,1726777,...,NaN,1,2021-08-30 18:39:46 UTC,13128549,0xf9b2bc7d6f7556852d2cdb742f92946d0b3677ee9574...,1.340000e+11,1.340000e+11,2.0,134000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936


In [24]:
complete_deposit_tx_df = pd.concat([wallet_to_tcash_deposit_df, wallet_to_proxy_deposits_df])

#### Finally, we get the complete deposit df

In [25]:
complete_deposit_tx_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address
0,0xcf97c470a56d96625c7240d3004ae2abd9141d7ffc43...,4,10,0xb050dec5a9010f8b77a3962369b7bc737d3ed4a5,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,1200000,56000000000,0xb214faa5083d52f2c4e7a95e340a999805a8328b6546...,1677906,...,NaN,1,2020-11-02 17:47:30 UTC,11179130,0x21d86cba454fea4f7e43c68763d4cffec101b6145546...,NaN,NaN,NaN,56000000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d
1,0x2fadc4d54bf6e3ec5035f14c0096befe929998663098...,22,118,0x6e92bc493c6abbdd6a1b18416f003de2c873ab50,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,1200000,63000000000,0xb214faa502843565a5499c26b04c7a9069f1ad2621aa...,8209080,...,NaN,1,2021-01-02 23:37:35 UTC,11578054,0xd52ababd1dbbbe50277c54473c881f47fd95e78081a3...,NaN,NaN,NaN,63000000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d
10,0x7baf0a76f35c1dece97fff883aa7174454bed460b1ba...,240,171,0x8c4c44fd06f7f98f08bf6a9ca156cec9ee1f31f8,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144,0,800000,105000000000,0xb214faa50afc4880238aac4198bceddea0fbf85e9ad3...,10723996,...,NaN,0,2021-01-06 19:04:40 UTC,11602841,0x7c5f21ea2a92f5182ce8648f152b6fb3b4379096309d...,NaN,NaN,NaN,105000000000,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144
11,0x7f851ba1d7292ca565961073a1111f7408760540c6f9...,0,84,0x50b9d4af009b038506d4d84b035c451d1a3a20bc,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,100000000000000000,800000,5000000000,0xb214faa51174c586ebae02a3b4f54a0ba6a9b7aba4d7...,8465851,...,NaN,0,2020-01-05 05:19:39 UTC,9218451,0x77e0fc60906714a54697cfd9275b622ad9d2d0271c4e...,NaN,NaN,NaN,5000000000,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
12,0xbd83053f8afa7777f54a4aca6b8e112fa31b888922dc...,7,63,0x6c6e4816ecfa4481472ff88f32a3e00f2eaa95a1,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,100000000000000000,800000,30838446643,0xb214faa527a20ba920c8ae877d67ce1ebd7420dafb31...,6222489,...,NaN,0,2020-05-27 03:30:44 UTC,10145408,0x837b3482443f027f6f045644bf002243f72304686015...,NaN,NaN,NaN,30838446643,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181260,0xb16040d97e3c75a137c2a826a2c5307d58612a0f881f...,19,382,0xb12814fdbbdc8c8f9c2741bde788813a90588be6,0x722122df12d4e14e13ac3b6895a86e84145b6967,100000000000000000000,1019350,81846727568,0x13d98d13000000000000000000000000a160cdab2256...,29897689,...,NaN,1,2021-09-07 03:08:40 UTC,13176145,0xf1b67ec807e6be55bc6995e31c3ee4bf8cb6e6610754...,8.255631e+10,1.220177e+09,2.0,81846727568,0xa160cdab225685da1d56aa342ad8841c3b53f291
181261,0x29d727360e68b35743d73d95f344ed885e2a9e33887f...,3,173,0xdcaaa4717c46840e6bdc1e067478210b0758055c,0x722122df12d4e14e13ac3b6895a86e84145b6967,100000000000000000000,200000,53892307010,0x13d98d13000000000000000000000000a160cdab2256...,17806851,...,NaN,0,2021-09-12 17:59:59 UTC,13212438,0xa50afdfc6c73d19a886575e40b8a88e8f00d76635a6e...,6.413296e+10,1.188590e+09,2.0,53892307010,0xa160cdab225685da1d56aa342ad8841c3b53f291
181265,0x9c5e4b8b996349237a9df035f9e38dbd69f3bb280ecc...,8,44,0x8cf22ec65677475aa41b0537b98ccee63ceca973,0x722122df12d4e14e13ac3b6895a86e84145b6967,10000000000000000000,1024570,71609345538,0x13d98d13000000000000000000000000910cbd523d97...,5911751,...,NaN,1,2021-08-29 00:56:36 UTC,13117282,0x2c3d7ef9d4e85085b72b86807278679e07b078d85996...,1.287044e+11,5.910977e+09,2.0,71609345538,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf
181266,0xeae7c8d21bb97391f21b7f3443ac51429092e0290fe6...,189,276,0x68661550f759a41d65a8eefd3e47dd606ca76ae2,0x722122df12d4e14e13ac3b6895a86e84145b6967,100000000000000000000,1008130,51523487883,0x13d98d13000000000000000000000000a160cdab2256...,28714246,...,NaN,1,2021-10-17 01:38:32 UTC,13432575,0xc59f24dbf6573c5c17f16d08657c78dea20377230f11...,6.838974e+10,1.301710e+09,2.0,51523487883,0xa

#### Now we have to do the same thing with the withdrawls transactions

Here we will have to make an extra step, as we need to differenciate between the relayer and recipient addresses, the same we already did here: 
https://github.com/lambdaclass/tornado_cash_anonymity_tool/blob/main/notebooks/complete_withdraw_data_set.ipynb

In [26]:
wallet_to_proxy_withdraws_df = withdraw_transactions_df[withdraw_transactions_df["to_address"].isin(["0x722122df12d4e14e13ac3b6895a86e84145b6967", "0x905b63fff465b9ffbf41dea908ceb12478ec7601"])]

In [27]:
wallet_to_tcash_withdraws_df = withdraw_transactions_df[withdraw_transactions_df["to_address"].isin(tornado_addresses_df["address"])]

In [28]:
# Here we find 17 transactions (over 83 k) that are no interacting with the proxy nor the contracts.
# These are mostly legacy contracts that have been killed or cont be decoded
withdraw_transactions_df[~withdraw_transactions_df["to_address"].isin(pd.concat([wallet_to_proxy_withdraws_df["to_address"], wallet_to_tcash_withdraws_df["to_address"]]))]

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
17307,0xcc9b675d85711fd1a673a5204dd75ed96fd7626fe513...,8,58,0x2d4add645f8cb8ab771342bb8ca1f7a4e0bdd263,0x7811377e0f81d97ab1890d1f8177107c930b0595,0,428890,1846153850,0x7e087fac00000000000000000000000012d66f87a04a...,3854056,...,NaN,NaN,1,2020-02-20 23:28:50 UTC,9523100,0x1122eed8d35abb665e0cc6dbea4f5eaa2bf6af545421...,NaN,NaN,NaN,1846153850
20048,0xce2b74f7eef75c6ed5267ab29bb0afa6a0ba00649603...,505,92,0x909ae1fc12c1c64f7554dc27a1c7785efaba79b7,0xb5fc86a50ad433fff6c2e3840409a3432858923f,100000000000000,500000,17486531365,0x2db80eb1000000000000000000000000000000000000...,6174691,...,NaN,NaN,1,2020-10-24 13:35:14 UTC,11119353,0xf46775d5cf4f43f407d50584ecb7c54748e004c276b5...,NaN,NaN,NaN,17486531365
20603,0x0336f15bd4ca20c9d27c089a3bd06cd4c4f147e7ac74...,172,99,0x1a776c84d64ecada985c968c3589b3c8615a1e7c,0xb5fc86a50ad433fff6c2e3840409a3432858923f,0,500000,17100000000,0x4fdc0c90000000000000000000000000000000000000...,11474390,...,NaN,NaN,1,2020-10-23 22:59:26 UTC,11115392,0xf092cee7c0e38743848e0c50e4a262a6a639568e10c5...,NaN,NaN,NaN,17100000000
26210,0x06daece183098afe6f3d2ad2ab1ab6fc52ac5b39c083...,891,134,0xa2bf60058c0657c45fdd1741220b4a7f0da91ca3,0xb5fc86a50ad433fff6c2e3840409a3432858923f,0,500000,58000000000,0x2db80eb1000000000000000000000000000000000000...,12117107,...,NaN,NaN,1,2020-10-29 16:50:25 UTC,11152849,0xe61598e6f9834029149f726cc24720b438a80721447e...,NaN,NaN,NaN,58000000000
42532,0xb9c05751f9ae7a9e4ccdcdf318d59d1040b3ea438976...,26,157,0xa2a01b4a68575280a2de45178e289da717bedb6f,0xb5fc86a50ad433fff6c2e3840409a3432858923f,0,500000,51000000000,0x7bd0d2af000000000000000000000000000000000000...,8638207,...,NaN,NaN,1,2020-10-23 17:43:06 UTC,11113994,0xe860ea5dcab0949fc95c4dfe4927d276deac3969773f...,NaN,NaN,NaN,51000000000
69473,0x7c0d70c86df1c783c6f59443f5119140e5e1d2682a4e...,154,191,0x1a776c84d64ecada985c968c3589b3c8615a1e7c,0x1beb3eb7fd4908cbcd3b0b62ffee06c98d0db43c,100000000000000,500000,40000000000,0x7bd0d2af000000000000000000000000000000000000...,10810055,...,NaN,NaN,0,2020-10-23 12:24:15 UTC,11112509,0x033d142468868ba999c57ec5eb8da32b06c0b91ed708...,NaN,NaN,NaN,40000000000
86886,0xa99ce9f6dacddf0ab456c5543f03965b6ae3a1191961...,155,138,0x1a776c84d64ecada985c968c3589b3c8615a1e7c,0x1beb3eb7fd4908cbcd3b0b62ffee06c98d0db43c,100000000000000,500000,44412087912,0x7bd0d2af000000000000000000000000000000000000...,7263246,...,NaN,NaN,0,2020-10-23 12:29:31 UTC,11112530,0x1fe85a3ccf259bf13d48ec7a254888fb9853b4edae17...,NaN,NaN,NaN,44412087912
87358,0xed2dfa830fb6afe481bf20baf5f27863744fff0a6ef3...,148,175,0x1a776c84d64ecada985c968c3589b3c8615a1e7c,0xcf146f5943ddcfdf8de4ef551d66ac7a8087324b,0,653091,35000000000,0x2db80eb1000000000000000000000000000000000000...,9519687,...,NaN,NaN,1,2020-10-22 20:02:49 UTC,11108105,0x17b5f5ecf2e585687d200a249270607646cad92cf6ec...,NaN,NaN,NaN,35000000000
94333,0x2237c22e71d094a29b4967afe7f9c95e2a0328d932d6...,1,6,0x20c62682cbe1eea82ef7a985b3805f5c360813ac,0x34410f5b9a1aaad11961b50a62c1d2913cd3a588,0,500000,0,0xb438689f00000000000000000000000047ce0c6ed5b0...,947778,...,NaN,NaN,1,2021-04-27 15:11:48 UTC,12323153,0x4751d27982b19a4b93ca54b3867f19e032e165d1492f...,NaN,NaN,NaN,0
112175,0xa0d81b1bbbd984c298e73bd9ab749d77ca43eb0efa09...,660,6,0xf3f7e999fd26566fa02040d9cd6506ca83d7f9b3,0xd216153c06e857cd7f72665e0af1d7d82172f494,0,2425164,10000000000,0x405cec670000000000000000000000007149173ed763...,603876,...,NaN,NaN,1,2020-03-03 01:14:10 UTC,9595147,0x080e526b8acdbae3cbfbf8f462ec4599ea2356e42161...,NaN,NaN,NaN,10000000000


#### Lets decode the Tcash contract that the proxy is going to call ("_tornado" field) and the recipient address that make tha withdraw

In [29]:
def tornado_address_withdraws_tor_address(row, tornado_contracts_dic):
    func_obj, func_params = tornado_contracts_dic[row["to_address"]].decode_function_input(row["input"])
    return func_params["_tornado"].lower()

In [30]:
def tornado_address_withdraws_recipient(row, tornado_contracts_dic):
    func_obj, func_params = tornado_contracts_dic[row["to_address"]].decode_function_input(row["input"])
    return func_params["_recipient"].lower()

In [31]:
wallet_to_proxy_withdraws_df["tornado_cash_address"] = wallet_to_proxy_withdraws_df.progress_apply(lambda row: tornado_address_withdraws_tor_address(row, proxy_contracts_dict), axis=1)

100%|██████████| 61773/61773 [01:11<00:00, 860.74it/s]
<ipython-input-31-dc87841c3439>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wallet_to_proxy_withdraws_df["tornado_cash_address"] = wallet_to_proxy_withdraws_df.progress_apply(lambda row: tornado_address_withdraws_tor_address(row, proxy_contracts_dict), axis=1)


In [32]:
wallet_to_proxy_withdraws_df["recipient_address"] = wallet_to_proxy_withdraws_df.progress_apply(lambda row: tornado_address_withdraws_recipient(row, proxy_contracts_dict), axis=1)

100%|██████████| 61773/61773 [01:10<00:00, 878.17it/s]
<ipython-input-32-33ae9636d03c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wallet_to_proxy_withdraws_df["recipient_address"] = wallet_to_proxy_withdraws_df.progress_apply(lambda row: tornado_address_withdraws_recipient(row, proxy_contracts_dict), axis=1)


In [33]:
wallet_to_proxy_withdraws_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address,recipient_address
2,0x9a2584e229b27d3bb87537e4cc4fb7b7cc085f968774...,6782,95,0xbe4d1e137a24af091be80ae58d652279665e3a27,0x722122df12d4e14e13ac3b6895a86e84145b6967,1650000000000000,480000,11500000000,0xb438689f00000000000000000000000007687e702b41...,7595061,...,1,2021-07-25 12:29:09 UTC,12895440,0x1f6c7c0cad03c9f00d421e096836a40794db70b056af...,NaN,NaN,NaN,11500000000,0x07687e702b410fa43f4cb4af7fa097918ffd2730,0x06ac83cec6b3596f5626c80e5076a4d408bd2ce1
3,0x2b40673c95b759b0967ac50185fe5691554819ac7e11...,1748,185,0x20bb3095a4852f4c97d7a188e9f7183c85acfc49,0x722122df12d4e14e13ac3b6895a86e84145b6967,3300000000000000,480000,23500000000,0xb438689f00000000000000000000000023773e65ed14...,11046553,...,1,2021-06-09 18:45:22 UTC,12601999,0xd653640a8a0fbd9da49b17e193db1773e3cb405cae1c...,NaN,NaN,NaN,23500000000,0x23773e65ed146a459791799d01336db287f25334,0xd4638a640d820a5f58bd27082cfe4d9d3ff3cbcc
4,0x34478d48b83ce20881013312c6edf130573e561aa0c6...,1963,88,0xdd36d585a0d32b4ae6f1790bc4292308e21a42e3,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,480000,13100000000,0xb438689f000000000000000000000000178169b423a0...,7194819,...,1,2021-07-23 10:50:49 UTC,12882215,0x217cd77b1e2f67bc3692c3874deca3c0bba12df926ed...,NaN,NaN,NaN,13100000000,0x178169b423a011fff22b9e3f3abea13414ddd0f1,0x774ff72b32ecd32f06b8e9254ff4a76bbc87ceca
5,0x532078c03eabcffce384d1aae7aa0444b1348eccbdd3...,1276,164,0x28a51fd7b39b49ad4d7813d89e3d4444d79e3ac6,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,420000,99000000000,0xb438689f000000000000000000000000610b717796ad...,8554775,...,1,2021-05-07 18:35:07 UTC,12388809,0x3dbc248d77885384863163034b21cffec41f4569b9fc...,NaN,NaN,NaN,99000000000,0x610b717796ad172b316836ac95a2ffad065ceab4,0xc08922a77140ce1aa91d419f4ac2ddc853575511
6,0xbf1d3100b2cb759e43aadd8777b08696144918b6402a...,2482,102,0x49136693081f2c18e2cf14428dd78cd90a22dc1f,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,480000,27616895407,0xb438689f000000000000000000000000610b717796ad...,7804781,...,1,2021-08-21 23:56:13 UTC,13071743,0xf7a2bd116556514b64369b726376831c5db0bccc95c6...,NaN,NaN,NaN,27616895407,0x610b717796ad172b316836ac95a2ffad065ceab4,0x774ff72b32ecd32f06b8e9254ff4a76bbc87ceca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181253,0x5a36df89fa1f1ef6de7254e4b5fbfd2d6c2297ef4621...,164,188,0xd2ddee09e723d165233986e0ed0aa6da1cea97e0,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,570447,70188284576,0xb438689f00000000000000000000000047ce0c6ed5b0...,20743650,...,1,2021-09-05 01:23:28 UTC,13162729,0xe14fb1085bc975af1ad751aa579d7de298fb5be86c3d...,1.027888e+11,2.210156e+09,2.0,70188284576,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x107753c195edde2fe69e573f667a697526178a47
181255,0x43a081968e109f71a2f807f4a94ffc77429b322c3c1e...,169,168,0xe7a784acb145bd8993e83d41adf43b7c7195e995,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,567923,39255417592,0xb438689f000000000000000000000000a160cdab2256...,16358129,...,1,2021-10-03 14:49:29 UTC,13346858,0xe5a4d9c829998f00873f6ac4603174d707ffb2ea79a3...,4.000000e+10,1.130179e+09,2.0,39255417592,0xa160cdab225685da1d56aa342ad8841c3b53f291,0xe7a784acb145bd8993e83d41adf43b7c7195e995
181262,0xd0708b3e2135d0378a9d6fe3fdc58b3c4fbcbcbe926c...,246,10,0xa85f4669b31ad9c75adbfd288c37b678ab0b0fa4,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,567971,50193886615,0xb438689f00000000000000000000000047ce0c6ed5b0...,982698,...,1,2021-10-17 23:36:24 UTC,13438417,0x36808310975d158e0f2b3a2f892abed25a955684f407...,7.518635e+10,1.519911e+09,2.0,50193886615,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0xa85f4669b31ad9c75adbfd288c37b678ab0b0fa4
181263,0x54b5ceca9fd3d2ff7893c32f901f3d1d5e2e4383607b...,127,180,0xd2ddee09e723d165233986e0ed0aa6da1cea97e0,0x722122df12d4e14

In [34]:
wallet_to_tcash_withdraws_df["tornado_cash_address"] = wallet_to_tcash_withdraws_df["to_address"]

<ipython-input-34-bd0c53e063b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wallet_to_tcash_withdraws_df["tornado_cash_address"] = wallet_to_tcash_withdraws_df["to_address"]


In [35]:
wallet_to_tcash_withdraws_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address
7,0x3dc8617176f45c0f33a95589c202b22f9ba8aa7067bb...,789,196,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0xa160cdab225685da1d56aa342ad8841c3b53f291,0,386384,121000000000,0x21a0adb6000000000000000000000000000000000000...,10382378,...,NaN,1,2020-10-05 07:57:28 UTC,10994498,0xa96abb653dd6d4e3cedd6bd65fd0a7b034e4c9527be9...,NaN,NaN,NaN,121000000000,0xa160cdab225685da1d56aa342ad8841c3b53f291
8,0x3199144ba98021e523c5ec1e4f4dcf64446d69b825da...,9,190,0xb77562124be8ac967cf7fc24573fe252aa39d95d,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13000000000,0x21a0adb6000000000000000000000000000000000000...,7303106,...,NaN,1,2020-02-11 19:10:54 UTC,9463504,0x1e1dc9cec82d1dcd0e68e60968d180408007e148ac06...,NaN,NaN,NaN,13000000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d
9,0x95a46a1305ddc8b7e01d82c7d6766fce4713c085929f...,2,81,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13200000000,0x21a0adb6000000000000000000000000000000000000...,3211601,...,NaN,1,2020-02-16 16:05:14 UTC,9495139,0x1f0562c98f40f0689ed8bac422b4714289bd452d9290...,NaN,NaN,NaN,13200000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d
13,0x172a0a1618aa6c1f9f27a370236d8c91156fd6192ad1...,2024,189,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,384581,40000000000,0x21a0adb6000000000000000000000000000000000000...,11795370,...,NaN,1,2020-11-09 17:08:00 UTC,11224582,0xe0ae8902495001fc28025238a74703c1b3d58282aaee...,NaN,NaN,NaN,40000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936
15,0x30becee80e63a039102df3b1243e5d314534abfcf3ed...,2350,127,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,404304,15020000842,0x21a0adb6000000000000000000000000000000000000...,9771737,...,NaN,1,2020-11-29 01:18:57 UTC,11350558,0x4d8f724d2bdffa546cef74d21c6c43968b566b67318e...,NaN,NaN,NaN,15020000842,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158865,0x6e9b9845946589d25bd3f1242f32b0e453419ff1086a...,464,100,0x49516e20b5692839f877a7e9aa62006a5d02a7b1,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,404352,78000000000,0x21a0adb6000000000000000000000000000000000000...,8278993,...,NaN,1,2020-11-17 14:04:20 UTC,11275872,0xd3debb8735a7857ad0fdf1c65c28083c332b69498448...,NaN,NaN,NaN,78000000000,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf
158866,0xd8988fe191bc13bc337833116b83f21a1358aa180541...,541,27,0x49516e20b5692839f877a7e9aa62006a5d02a7b1,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,406095,45625000000,0x21a0adb6000000000000000000000000000000000000...,2295663,...,NaN,1,2020-11-23 20:41:11 UTC,11316682,0x5943fedd002ffc877f9c3cd0841701f67ed81db0492f...,NaN,NaN,NaN,45625000000,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf
158867,0x2431dfaaf2cab467fe8b09170a3c08f7c288a447e3a2...,22,102,0x3750b281e14b0fcd4880399210ce03a7f570fbd7,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,404352,35000000000,0x21a0adb6000000000000000000000000000000000000...,8717725,...,NaN,1,2020-11-18 22:17:43 UTC,11284563,0x153246c07dad60b459880157fbad0066cb007187d354...,NaN,NaN,NaN,35000000000,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf
158868,0xfc319bb6d83fffba3c9558f743cecf84cd8743ea3bbc...,886,61,0x0360b232285a801cbae01c984fb680ba2f465fdc,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,406095,50000000000,0x21a0adb6000000000000000000000000000000000000...,3752579,...,NaN,1,2020-10-01 22:31:58 UTC,10972896,0x17764be631d631fa2ceae77feed560ba94452aace23c...,NaN,NaN,NaN,50000000000,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf


In [43]:
tornado_contract_dict = tornado_contracts(tornado_addresses_df)


In [37]:
wallet_to_tcash_withdraws_df["recipient_address"] = wallet_to_tcash_withdraws_df.progress_apply(lambda row: tornado_address_withdraws_recipient(row, tornado_contract_dict), axis=1)

100%|██████████| 22009/22009 [00:39<00:00, 562.37it/s]
<ipython-input-37-1fcab8beb493>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wallet_to_tcash_withdraws_df["recipient_address"] = wallet_to_tcash_withdraws_df.progress_apply(lambda row: tornado_address_withdraws_recipient(row, tornado_contract_dict), axis=1)


In [38]:
wallet_to_tcash_withdraws_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address,recipient_address
7,0x3dc8617176f45c0f33a95589c202b22f9ba8aa7067bb...,789,196,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0xa160cdab225685da1d56aa342ad8841c3b53f291,0,386384,121000000000,0x21a0adb6000000000000000000000000000000000000...,10382378,...,1,2020-10-05 07:57:28 UTC,10994498,0xa96abb653dd6d4e3cedd6bd65fd0a7b034e4c9527be9...,NaN,NaN,NaN,121000000000,0xa160cdab225685da1d56aa342ad8841c3b53f291,0xd843ce0f9da3bd30537b9850711ab8089e06b4cf
8,0x3199144ba98021e523c5ec1e4f4dcf64446d69b825da...,9,190,0xb77562124be8ac967cf7fc24573fe252aa39d95d,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13000000000,0x21a0adb6000000000000000000000000000000000000...,7303106,...,1,2020-02-11 19:10:54 UTC,9463504,0x1e1dc9cec82d1dcd0e68e60968d180408007e148ac06...,NaN,NaN,NaN,13000000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0xb77562124be8ac967cf7fc24573fe252aa39d95d
9,0x95a46a1305ddc8b7e01d82c7d6766fce4713c085929f...,2,81,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13200000000,0x21a0adb6000000000000000000000000000000000000...,3211601,...,1,2020-02-16 16:05:14 UTC,9495139,0x1f0562c98f40f0689ed8bac422b4714289bd452d9290...,NaN,NaN,NaN,13200000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c
13,0x172a0a1618aa6c1f9f27a370236d8c91156fd6192ad1...,2024,189,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,384581,40000000000,0x21a0adb6000000000000000000000000000000000000...,11795370,...,1,2020-11-09 17:08:00 UTC,11224582,0xe0ae8902495001fc28025238a74703c1b3d58282aaee...,NaN,NaN,NaN,40000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x42a9cf901e889ca8a685394854c27898dbff86b0
15,0x30becee80e63a039102df3b1243e5d314534abfcf3ed...,2350,127,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,404304,15020000842,0x21a0adb6000000000000000000000000000000000000...,9771737,...,1,2020-11-29 01:18:57 UTC,11350558,0x4d8f724d2bdffa546cef74d21c6c43968b566b67318e...,NaN,NaN,NaN,15020000842,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0xc9fb4b16800e61076326145e1066e876cf9f7a7b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158865,0x6e9b9845946589d25bd3f1242f32b0e453419ff1086a...,464,100,0x49516e20b5692839f877a7e9aa62006a5d02a7b1,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,404352,78000000000,0x21a0adb6000000000000000000000000000000000000...,8278993,...,1,2020-11-17 14:04:20 UTC,11275872,0xd3debb8735a7857ad0fdf1c65c28083c332b69498448...,NaN,NaN,NaN,78000000000,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0x6283511fc53c60c0a0a1848ac88ff46b5d2496ea
158866,0xd8988fe191bc13bc337833116b83f21a1358aa180541...,541,27,0x49516e20b5692839f877a7e9aa62006a5d02a7b1,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,406095,45625000000,0x21a0adb6000000000000000000000000000000000000...,2295663,...,1,2020-11-23 20:41:11 UTC,11316682,0x5943fedd002ffc877f9c3cd0841701f67ed81db0492f...,NaN,NaN,NaN,45625000000,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0xb426441f72c8ce9ab93f408e364d884b2f93cb14
158867,0x2431dfaaf2cab467fe8b09170a3c08f7c288a447e3a2...,22,102,0x3750b281e14b0fcd4880399210ce03a7f570fbd7,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,404352,35000000000,0x21a0adb6000000000000000000000000000000000000...,8717725,...,1,2020-11-18 22:17:43 UTC,11284563,0x153246c07dad60b459880157fbad0066cb007187d354...,NaN,NaN,NaN,35000000000,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0x7c24325df911276189fccddc3977fdadb8e72d0d
158868,0xfc319bb6d83fffba3c9558f743cecf84cd8743ea3bbc...,886,61,0x0360b232285a801cbae01c984fb680ba2f465fdc,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,406095,50000000000,0x21a0adb6000000000000000000000000000000000

In [39]:
complete_withdraw_tx_df = pd.concat([wallet_to_tcash_withdraws_df, wallet_to_proxy_withdraws_df])

In [40]:
complete_withdraw_tx_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address,recipient_address
7,0x3dc8617176f45c0f33a95589c202b22f9ba8aa7067bb...,789,196,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0xa160cdab225685da1d56aa342ad8841c3b53f291,0,386384,121000000000,0x21a0adb6000000000000000000000000000000000000...,10382378,...,1,2020-10-05 07:57:28 UTC,10994498,0xa96abb653dd6d4e3cedd6bd65fd0a7b034e4c9527be9...,NaN,NaN,NaN,121000000000,0xa160cdab225685da1d56aa342ad8841c3b53f291,0xd843ce0f9da3bd30537b9850711ab8089e06b4cf
8,0x3199144ba98021e523c5ec1e4f4dcf64446d69b825da...,9,190,0xb77562124be8ac967cf7fc24573fe252aa39d95d,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13000000000,0x21a0adb6000000000000000000000000000000000000...,7303106,...,1,2020-02-11 19:10:54 UTC,9463504,0x1e1dc9cec82d1dcd0e68e60968d180408007e148ac06...,NaN,NaN,NaN,13000000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0xb77562124be8ac967cf7fc24573fe252aa39d95d
9,0x95a46a1305ddc8b7e01d82c7d6766fce4713c085929f...,2,81,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13200000000,0x21a0adb6000000000000000000000000000000000000...,3211601,...,1,2020-02-16 16:05:14 UTC,9495139,0x1f0562c98f40f0689ed8bac422b4714289bd452d9290...,NaN,NaN,NaN,13200000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c
13,0x172a0a1618aa6c1f9f27a370236d8c91156fd6192ad1...,2024,189,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,384581,40000000000,0x21a0adb6000000000000000000000000000000000000...,11795370,...,1,2020-11-09 17:08:00 UTC,11224582,0xe0ae8902495001fc28025238a74703c1b3d58282aaee...,NaN,NaN,NaN,40000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x42a9cf901e889ca8a685394854c27898dbff86b0
15,0x30becee80e63a039102df3b1243e5d314534abfcf3ed...,2350,127,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,404304,15020000842,0x21a0adb6000000000000000000000000000000000000...,9771737,...,1,2020-11-29 01:18:57 UTC,11350558,0x4d8f724d2bdffa546cef74d21c6c43968b566b67318e...,NaN,NaN,NaN,15020000842,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0xc9fb4b16800e61076326145e1066e876cf9f7a7b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181253,0x5a36df89fa1f1ef6de7254e4b5fbfd2d6c2297ef4621...,164,188,0xd2ddee09e723d165233986e0ed0aa6da1cea97e0,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,570447,70188284576,0xb438689f00000000000000000000000047ce0c6ed5b0...,20743650,...,1,2021-09-05 01:23:28 UTC,13162729,0xe14fb1085bc975af1ad751aa579d7de298fb5be86c3d...,1.027888e+11,2.210156e+09,2.0,70188284576,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x107753c195edde2fe69e573f667a697526178a47
181255,0x43a081968e109f71a2f807f4a94ffc77429b322c3c1e...,169,168,0xe7a784acb145bd8993e83d41adf43b7c7195e995,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,567923,39255417592,0xb438689f000000000000000000000000a160cdab2256...,16358129,...,1,2021-10-03 14:49:29 UTC,13346858,0xe5a4d9c829998f00873f6ac4603174d707ffb2ea79a3...,4.000000e+10,1.130179e+09,2.0,39255417592,0xa160cdab225685da1d56aa342ad8841c3b53f291,0xe7a784acb145bd8993e83d41adf43b7c7195e995
181262,0xd0708b3e2135d0378a9d6fe3fdc58b3c4fbcbcbe926c...,246,10,0xa85f4669b31ad9c75adbfd288c37b678ab0b0fa4,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,567971,50193886615,0xb438689f00000000000000000000000047ce0c6ed5b0...,982698,...,1,2021-10-17 23:36:24 UTC,13438417,0x36808310975d158e0f2b3a2f892abed25a955684f407...,7.518635e+10,1.519911e+09,2.0,50193886615,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0xa85f4669b31ad9c75adbfd288c37b678ab0b0fa4
181263,0x54b5ceca9fd3d2ff7893c32f901f3d1d5e2e4383607b...,127,180,0xd2ddee09e723d165233986e0ed0aa6da1cea97e0,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,57045

In [41]:
# complete_withdraw_tx_df.to_csv("../data/complete_withdraw_txs.csv")

In [42]:
# complete_deposit_tx_df.to_csv("../data/complete_deposit_txs.csv")